In [1]:
# configurations

from file_paths import *
import random

model_and_data={
    "allenai/OLMo-2-1124-7B":[AS_cpp, AS_python, AS_github_lean,AS_julia,AS_tex, AS_github_isabelle,AS_fortran, AS_github_coq, AS_r],
    'HuggingFaceH4/zephyr-7b-beta':[UltraChat],
    BioMistral:[RE_2012temporal,STS_STS_B,DC_MTSample,RE_2011coreference,events_BioRed, events_NLM_Gene,events_2012temporal,events_2006deid,events_BioASQ],
    'EleutherAI/pythia-70m':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    'EleutherAI/pythia-160m':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    'EleutherAI/pythia-410m':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    'EleutherAI/pythia-1.4b':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    'EleutherAI/pythia-2.8b':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    'EleutherAI/pythia-6.9b':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    'EleutherAI/pythia-12b':[Github,FreeLaw,Enron_Emails,ArXiv,OpenWeb_Text2,Open_Subtitles,Hacker_News,YoutubeSubtitles,Pile_CC],
    }


# step 1 getting the perplexity, token ranks and token probabilities for each instance.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specifying the GPU cores you want to use.
os.environ["WORLD_SIZE"] = "2"
from probability_helper import *
assert torch.cuda.is_available()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

for base_model_id in model_and_data:
    model,tokenizer=load_model(base_model_id,device)
    if 'BioMistral' in base_model_id:
        model_name='BioMistral' 
    else:
        model_name=base_model_id.split('/')[-1]
    
    outdir=output_path+"/"+model_name+'/pairs'
    domains=model_and_data[base_model_id]
    for domain in domains:
        try:
            train_file,test_file=domain
            if 'BioMistral' in model_name:
                domain_name='_'.join(train_file.split('/')[-3:-1]).replace('-','_')
            else:
                domain_name=train_file.split('/')[-1].replace('_train.jsonl','').replace('train.json','')
            
            for key,filename in [['unseen',test_file],['seen',train_file]]:
                outname=f'{outdir}/{domain_name}_{key}.json'
                if os.path.isfile(outname):
                    continue
                if 'jsonl' in filename:
                    all_lines=open(filename,encoding='utf-8').readlines()
                    if len(all_lines)>1101:
                        random.seed(0)
                        all_lines=random.sample(all_lines,1100)
                    dics=[json.loads(line) for line in all_lines][:1000]
                else:
                    dics=json.loads(open(filename,encoding='utf-8').read())
                    if len(dics)>1000:
                        dics=random.sample(dics,1000)
                output=[]
                for dic in tqdm(dics):
                    if 'text' in dic:
                        text=dic["text"]
                    elif 'messages' in dic:
                        text='\n'.join([f'<{message["role"]}> {message["content"]} <{message["role"]}/>' for message in dic['messages']])
                        
                    offset_mapping,all_tokens,ranks,token_probs,perplexity=find_ranks(text,base_model_id,model,tokenizer,device, start_token=1)
                    output.append({
                        'text':text,
                        'tokens':all_tokens[:min(len(all_tokens),2048)],
                        'ranks':ranks,
                        'perplexity':perplexity,
                        'token_probs':token_probs,
                    })
                    #break
                os.makedirs(outdir,exist_ok=True)
                with open(outname,'w') as f:
                    json.dump(output,f,indent=4)
        except:
            print(f'{model}, {domain} failed')

# Step 2 calculating AUCs and plot density plots

In [ ]:
entropy_ps=[5, 10, 25]
rank_thresholds=[1,3,5] 
token_thresholds=[5,15,25]
ppl_keys=[f'ppl_{i}' for i in [50,100,200]]+['perplexity'] # 
rows=ppl_keys  +  [f'Min {k}% token' for k in token_thresholds] + [f'Mem {k}' for k in rank_thresholds] + [f'Entropy {k}' for k in entropy_ps] + ['PPL_seen','PPL_unseen']

domain_map={
    'RE_2012temporal':'RE 2012temp',
    'STS_STS_B':'STS B',
    'DC_MTSample':'DC MTSample', 
    'RE_2011coreference':'RE 2011coref',
    'events_BioRed':'events BioRed', 
    'events_NLM_Gene':'events NLMGene', 
    'events_2012temporal':'events 2012temp', 
    'events_2006deid':'events 2006deid', 
    'events_BioASQ':'events BioASQ'
}


density_set=set(['Github','Pile-CC'])
for model, files in model_and_data.items():
    if 'pythia-6.9b' not in model:
        continue
    if 'BioMistral' in model:
        domains=['_'.join(file[1].split('/')[-3:-1]).replace('-','_') for file in files]
    else:
        domains=[file[0].replace('/train.json','').replace('/evaluation.json','').split('/')[-1].split('_train')[0] for file in files]
    model_tag=model.split('/')[-1]
    outname=f'data/analysis/pairs/{model_tag}.json'
    if not os.path.isfile(outname):
        scores={}
        for domain1 in tqdm(domains):
            for domain2 in domains:
                train_file=f'data/{model_tag}/pairs/{domain1}_seen.json'
                test_file=f'data/{model_tag}/pairs/{domain2}_unseen.json'
                
                assert os.path.isfile(train_file),train_file
                assert os.path.isfile(test_file),test_file
                
                train_domain=domain_map.get(domain1,domain1)
                test_domain=domain_map.get(domain2,domain2)
                
                domain_set=set([train_domain,test_domain])
                plot_figure=model_tag=='pythia-6.9b' and (density_set.issuperset(domain_set) or domain_set==density_set )
                score=plot_and_calculate(train_file,test_file,train_domain,test_domain,model_tag,plot_figure=plot_figure)
                scores[f'{domain1}[sep]{domain2}']=score
        with open(outname,'w') as f:
            json.dump(scores,f,indent=4)

# step 3 save results to latex

In [ ]:
# saving those information to latex
for model, files in model_and_data.items():
    if 'BioMistral' in model:
        domains=['_'.join(file[1].split('/')[-3:-1]).replace('-','_') for file in files]
    else:
        domains=[file[0].replace('/train.json','').replace('/evaluation.json','').split('/')[-1].split('_train')[0] for file in files]
    model_tag=model.split('/')[-1]
    scores=json.load(open(f'data/analysis/pairs/{model_tag}.json'))
    model_tag=model.split('/')[-1]
    output=text_before
    for domain in domains:
        domain_tag=domain_map.get(domain,domain)
        words=domain_tag.split(' ')
        if len(words)==1:
            output+='& \\textbf{'+domain_tag+'}'
        else:
            output+='\\textbf{\\begin{tabular}[c]{@{}c@{}}'+words[0]+'-\\\\ '+words[1]+'\\end{tabular}}'
    output+='\\\\\hline\n'
    for line_start, metric in latex_lines.items():
        output+=line_start
        for domain in domains:
            score=scores[domain+"[sep]"+domain][metric]
            if isinstance(score,list):
                output+=f'& {round(score[0],1)}$\\pm${round(score[1],1)} '
            else:
                output+=f'& {round(score,1)} '
        if line_start==' & Entropy 25':
            output+='\\\\\hline\n\\multicolumn{2}{l}{\\textbf{Average AUC}}'
            for domain in domains:
                score=np.mean([scores[domain+"[sep]"+domain][m] for m in rows[:-2]])
                output+=f'& {round(score,1)} '
        output+='\n'
    if 'pythia' in model:
        dataset='Pile'
    elif 'mistral' in model.lower():
        dataset='Medical-NLU'
    else:
        dataset='Algebraic Stack'
    output+=get_text_after(model_tag,dataset)
    with open(f'data/analysis/latex/{model_tag}.tex','w') as f:
         f.write(output)

# step 4 cross-domain heatmap

In [ ]:
for model, files in model_and_data.items():
    if len(files)!=9 or 'pythia-6.9b' not in model:
        continue
    
    if 'BioMistral' in model:
        domains=['_'.join(file[1].split('/')[-3:-1]).replace('-','_') for file in files]
    else:
        domains=[file[0].replace('/train.json','').replace('/evaluation.json','').split('/')[-1].split('_train')[0] for file in files]
    
    abbreviations=[domain_map.get(domain1,domain1).split(' ') for domain in domains]
    abbreviations=[f'{word[0]}{word[1]}'.upper() if len(word)==2 else word[0][:4] for word in abbreviations]
    model_tag=model.split('/')[-1]
    model='6.9b'
    scores=json.load(open(f'data/analysis/pairs/{model_tag}.json'))
    for metric in rows[:-2]:
        data=[]
        for domain1 in domains: #seen
            values=[scores[f'{domain1}[sep]{domain2}'][metric] for domain2 in domains]
            data.append(values)
        if 'PPL' in metric:
            metric=metric.lower()
        outname=f'data/analysis/plots/heats/{model}_{metric}.png'.replace('%','').replace(' ','_')
        #summary.to_csv(outname+".csv")
        plot_heat_map(data,abbreviations,abbreviations,outname)
        #break